In [1]:
!apt-get update
!apt-get install -y chromium-browser
!apt install chromium-chromedriver

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-browser is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 17 not up

In [2]:
from selenium import webdriver
import re
import pandas as pd

In [3]:
!ls /usr/lib/chromium-browser/chromedriver

/usr/lib/chromium-browser/chromedriver


In [4]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [5]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select


In [6]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.headless = True
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://store.steampowered.com/charts/mostplayed")

In [7]:
from selenium.webdriver.common.action_chains import ActionChains


In [8]:
# all_elements = driver.find_elements(By.XPATH, "/html/body/div[1]/div[7]/div[6]/div[3]/div[3]/div/div/div/div[3]/table/tbody/tr[1]")

# # Iterate over the elements and get their class names
# class_names = []
# for element in all_elements:
#     class_name = element.get_attribute("class")
#     if class_name:
#         class_names.append(class_name)
# print(class_names)

In [9]:
class_name_of_rank_game = "weeklytopsellers_RankCell_34h48"
class_name_of_link_game = "weeklytopsellers_TopChartItem_2C5PJ"
class_name_of_name_game = "weeklytopsellers_GameName_1n_4-"
class_name_of_price_game = "weeklytopsellers_PriceCell_3IyfU"
class_name_of_current_player = "weeklytopsellers_ConcurrentCell_3L0CD"
class_name_of_peak_player = "weeklytopsellers_PeakInGameCell_yJB7D"

In [10]:
# rank_game_elements = driver.find_elements(By.CLASS_NAME, class_name_of_rank_game)
# link_game_elements = driver.find_elements(By.CLASS_NAME, class_name_of_link_game)
# name_game_elements = driver.find_elements(By.CLASS_NAME, class_name_of_name_game)
# price_game_elements = driver.find_elements(By.CLASS_NAME, class_name_of_price_game)
# current_player_elements = driver.find_elements(By.CLASS_NAME, class_name_of_current_player)
# peak_player_elements = driver.find_elements(By.CLASS_NAME, class_name_of_peak_player)

In [11]:
# rank_game_elements = driver.find_elements(By.CLASS_NAME, class_name_of_rank_game)
rank_games=[]
while len(rank_games)==0:
    rank_games = [element.text for element in driver.find_elements(By.CLASS_NAME, class_name_of_rank_game)]
    name_games = [element.text for element in driver.find_elements(By.CLASS_NAME, class_name_of_name_game)]
    price_games = [element.text for element in driver.find_elements(By.CLASS_NAME, class_name_of_price_game)]
    current_players = [element.text for element in driver.find_elements(By.CLASS_NAME, class_name_of_current_player)]
    peak_players = [element.text for element in driver.find_elements(By.CLASS_NAME, class_name_of_peak_player)]
    link_games = [element.get_attribute("href") for element in  driver.find_elements(By.CLASS_NAME, class_name_of_link_game)]

In [12]:
print(rank_games)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100']


In [13]:
data = {"rank": rank_games,
        "name": name_games,
        "price": price_games,
        "current_player": current_players,
        "peak_player": peak_players,
        "link": link_games
        }
df = pd.DataFrame(data)

In [14]:
df.head(10)

,rank,name,price,current_player,peak_player,link
0,1,Counter-Strike 2,Free To Play,"584,957","1,239,472",https://store.steampowered.com/app/730/Counter...
1,2,Dota 2,Free To Play,"277,172","734,790",https://store.steampowered.com/app/570/Dota_2?...
2,3,Baldur's Gate 3,$59.99,"194,411","258,655",https://store.steampowered.com/app/1086940/Bal...
3,4,Call of Duty®,,"115,493","161,126",https://store.steampowered.com/app/1938090/Cal...
4,5,Apex Legends™,Free To Play,"93,991","410,357",https://store.steampowered.com/app/1172470/Ape...
5,6,Team Fortress 2,Free To Play,"92,357","126,766",https://store.steampowered.com/app/440/Team_Fo...
6,7,Cyberpunk 2077,$59.99,"82,685","150,549",https://store.steampowered.com/app/1091500/Cyb...
7,8,Tom Clancy's Rainbow Six® Siege,-60%\n$19.99\n$7.99,"67,447","90,169",https://store.steampowered.com/app/359550/Tom_...
8,9,PUBG: BATTLEGROUNDS,Free To Play,"61,853","440,192",https://store.steampowered.com/app/578080/PUBG...
9,10,Starfield,$69.99,"59,792","82,141",https://store.steampowered.com/app/1716740/Sta...


In [15]:
# Close the browser
driver.quit()

In [16]:
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [17]:
def check_exists_by_xpath(webdriver,xpath):
    try:
        webdriver.find_element(By.XPATH,xpath)
    except NoSuchElementException:
        return False
    return True

def check_exists_by_class(webdriver,classname):
    try:
        webdriver.find_element(By.CLASS_NAME,classname)
    except NoSuchElementException:
        return False
    return True

In [18]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.headless = True
driver = webdriver.Chrome(options=chrome_options)

In [19]:
check_xpath='/html/body/div[1]/div[7]/div[6]/div/div[2]/div/div[1]/div[2]/select[3]'

publisher_selector='#game_highlights > div.rightcol > div > div.glance_ctn_responsive_left > div:nth-child(4) > div.summary.column > a'
system_requirements_xpath='/html/body/div[1]/div[7]/div[6]/div[3]/div[2]/div[1]/div[5]/div[2]/div[8]/div[1]/div[1]'

os_system_requirements_xpath='/html/body/div[1]/div[7]/div[6]/div[3]/div[2]/div[1]/div[5]/div[2]/div[8]/div[1]/div[1]/div[1]'

In [20]:
# driver.get(df.link[4])
# if check_exists_by_xpath(driver,check_xpath):
#   option=driver.find_element(By.XPATH,'/html/body/div[1]/div[7]/div[6]/div/div[2]/div/div[1]/div[2]/select[3]')
#   option.click()
#   option=driver.find_element(By.XPATH,'/html/body/div[1]/div[7]/div[6]/div/div[2]/div/div[1]/div[2]/select[3]/option[103]')
#   option.click()
#   option=driver.find_element(By.XPATH,'/html/body/div[1]/div[7]/div[6]/div/div[2]/div/div[1]/div[3]/div/a[1]')
#   option.click()

# name=df.name[4]
# genre=[element.text for element in driver.find_elements(By.CSS_SELECTOR, '#genresAndManufacturer > span')]
# feature_list=[element.text for element in driver.find_elements(By.CLASS_NAME, 'label')]
# if check_exists_by_class(driver,'all_languages'):
#   all_languages=driver.find_element(By.CLASS_NAME,'all_languages')
#   all_languages.click()
# supported_languages=[element.text for element in driver.find_elements(By.CLASS_NAME, 'ellipsis')]
# description=driver.find_element(By.CLASS_NAME, 'game_description_snippet').text
# num_curators=driver.find_element(By.CLASS_NAME, 'no_curators_followed').text

# wait = WebDriverWait(driver, 10)
# element = wait.until(EC.presence_of_element_located((By.CLASS_NAME,'user_reviews_filter_menu')))
# elm=driver.find_element(By.CLASS_NAME,'user_reviews_filter_menu')
# action=ActionChains(driver)
# action.move_to_element(elm)
# action.perform()
# num_reviews=['']
# while len(num_reviews[0])==0:
#   action.move_to_element(elm)
#   action.perform()
#   num_reviews=[element.text for element in driver.find_elements(By.CSS_SELECTOR, '#reviews_filter_options > div:nth-child(1) > div.user_reviews_filter_menu_flyout > div')]

# release_date=driver.find_element(By.CLASS_NAME, 'date').text
# developer=driver.find_element(By.ID, 'developers_list').text
# publisher=driver.find_element(By.CSS_SELECTOR, publisher_selector).text
# print(f'{name}\t{genre}')
# print(feature_list)
# print(supported_languages)
# print(description)
# print(num_curators)
# print(num_reviews)
# print(release_date)
# print(developer)
# print(publisher)


# if check_exists_by_xpath(driver,os_system_requirements_xpath):
#   system_requirements=[element.text for element in driver.find_elements(By.XPATH, system_requirements_xpath)]
#   l=len(system_requirements[0].split('\n'))
#   cnt=1
#   while cnt<=l:
#     os_system_requirements_xpath=f'/html/body/div[1]/div[7]/div[6]/div[3]/div[2]/div[1]/div[5]/div[2]/div[8]/div[1]/div[1]/div[{cnt}]'
#     os_system_requirements=driver.find_element(By.XPATH, os_system_requirements_xpath)
#     action.click(os_system_requirements).perform()
#     print(f'{os_system_requirements.text}: ')

#     detail_requirements_xpath=f'/html/body/div[1]/div[7]/div[6]/div[3]/div[2]/div[1]/div[5]/div[2]/div[8]/div[1]/div[2]/div[{cnt}]/div[1]'
#     detail_requirements=driver.find_element(By.XPATH, detail_requirements_xpath).text
#     print(detail_requirements)
#     cnt+=1
#     print()
# else:
#     single_os_requirements=driver.find_element(By.CLASS_NAME,'sysreq_contents').text
#     print(single_os_requirements)



In [21]:
list_game_detail=[]


In [22]:
miss=[]
for i in range(len(df)):
  game_detail={}
  name=[]
  try:
    driver.get(df.link[i])
    if check_exists_by_xpath(driver,check_xpath):
      option=driver.find_element(By.XPATH,'/html/body/div[1]/div[7]/div[6]/div/div[2]/div/div[1]/div[2]/select[3]')
      option.click()
      option=driver.find_element(By.XPATH,'/html/body/div[1]/div[7]/div[6]/div/div[2]/div/div[1]/div[2]/select[3]/option[103]')
      option.click()
      option=driver.find_element(By.XPATH,'/html/body/div[1]/div[7]/div[6]/div/div[2]/div/div[1]/div[3]/div/a[1]')
      option.click()

    name=df.name[i]
    genre=[element.text for element in driver.find_elements(By.CSS_SELECTOR, '#genresAndManufacturer > span')]
    feature_list=[element.text for element in driver.find_elements(By.CLASS_NAME, 'label')]

    if check_exists_by_class(driver,'all_languages'):
      all_languages=driver.find_element(By.CLASS_NAME,'all_languages')
      all_languages.click()
    supported_languages=[element.text for element in driver.find_elements(By.CLASS_NAME, 'ellipsis')]

    description=driver.find_element(By.CLASS_NAME, 'game_description_snippet').text

    release_date=driver.find_element(By.CLASS_NAME, 'date').text
    developer=driver.find_element(By.ID, 'developers_list').text
    publisher=driver.find_element(By.CSS_SELECTOR, publisher_selector).text

    num_curators=driver.find_element(By.CLASS_NAME, 'no_curators_followed').text


    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.presence_of_element_located((By.CLASS_NAME,'user_reviews_filter_menu')))
    elm=driver.find_element(By.CLASS_NAME,'user_reviews_filter_menu')
    num_reviews=[]
    action=ActionChains(driver)


    num_reviews=['']
    while len(num_reviews[0])==0:
      action.move_to_element(elm)
      action.perform()
      num_reviews=[element.text for element in driver.find_elements(By.CSS_SELECTOR, '#reviews_filter_options > div:nth-child(1) > div.user_reviews_filter_menu_flyout > div')]


    requirements=[]
    if check_exists_by_xpath(driver,os_system_requirements_xpath):
      system_requirements=[element.text for element in driver.find_elements(By.XPATH, system_requirements_xpath)]
      l=len(system_requirements[0].split('\n'))
      cnt=1
      while cnt<=l:
        os_system_requirements_xpath=f'/html/body/div[1]/div[7]/div[6]/div[3]/div[2]/div[1]/div[5]/div[2]/div[8]/div[1]/div[1]/div[{cnt}]'
        os_system_requirements=driver.find_element(By.XPATH, os_system_requirements_xpath)
        action.click(os_system_requirements).perform()
        detail_requirements_xpath=f'/html/body/div[1]/div[7]/div[6]/div[3]/div[2]/div[1]/div[5]/div[2]/div[8]/div[1]/div[2]/div[{cnt}]/div[1]'
        detail_requirements=driver.find_element(By.XPATH, detail_requirements_xpath).text
        requirements.append(detail_requirements)
        cnt+=1
    else:
        single_os_requirements=driver.find_element(By.CLASS_NAME,'sysreq_contents').text
        requirements.append(single_os_requirements)

    game_detail={'name':name,
                  'genre':genre,
                  'description': description,
                  'feature_list':feature_list,
                  'supported_languages':supported_languages,
                  'release_date':release_date,
                  'developer':developer,
                  'publisher':publisher,
                  'num_curators':num_curators,
                  'num_reviews':num_reviews,
                  'requirements':requirements
                  }
    print(game_detail['genre'])
    list_game_detail.append(game_detail)
    print(list_game_detail[i])
  except:
    try:
      error=driver.find_element(By.ID,'error_box').text
      print(error)
    except:
      miss.append(i)
      pass


['Action, Free to Play']
{'name': 'Counter-Strike 2', 'genre': ['Action, Free to Play'], 'description': 'For over two decades, Counter-Strike has offered an elite competitive experience, one shaped by millions of players from across the globe. And now the next chapter in the CS story is about to begin. This is Counter-Strike 2.', 'feature_list': ['Cross-Platform Multiplayer', 'Steam Trading Cards', 'Steam Workshop', 'In-App Purchases', 'Valve Anti-Cheat enabled', 'Stats', 'Remote Play on Phone', 'Remote Play on Tablet', 'Remote Play on TV'], 'supported_languages': ['English', 'Czech', 'Danish', 'Dutch', 'Finnish', 'French', 'German', 'Hungarian', 'Italian', 'Japanese', 'Korean', 'Norwegian', 'Polish', 'Portuguese - Portugal', 'Portuguese - Brazil', 'Romanian', 'Russian', 'Simplified Chinese', 'Spanish - Spain', 'Swedish', 'Thai', 'Traditional Chinese', 'Turkish', 'Bulgarian', 'Ukrainian', 'Greek', 'Spanish - Latin America', 'Vietnamese'], 'release_date': 'Aug 21, 2012', 'developer': 'V

In [23]:
list_game_detail

[{'name': 'Counter-Strike 2',
  'genre': ['Action, Free to Play'],
  'description': 'For over two decades, Counter-Strike has offered an elite competitive experience, one shaped by millions of players from across the globe. And now the next chapter in the CS story is about to begin. This is Counter-Strike 2.',
  'feature_list': ['Cross-Platform Multiplayer',
   'Steam Trading Cards',
   'Steam Workshop',
   'In-App Purchases',
   'Valve Anti-Cheat enabled',
   'Stats',
   'Remote Play on Phone',
   'Remote Play on Tablet',
   'Remote Play on TV'],
  'supported_languages': ['English',
   'Czech',
   'Danish',
   'Dutch',
   'Finnish',
   'French',
   'German',
   'Hungarian',
   'Italian',
   'Japanese',
   'Korean',
   'Norwegian',
   'Polish',
   'Portuguese - Portugal',
   'Portuguese - Brazil',
   'Romanian',
   'Russian',
   'Simplified Chinese',
   'Spanish - Spain',
   'Swedish',
   'Thai',
   'Traditional Chinese',
   'Turkish',
   'Bulgarian',
   'Ukrainian',
   'Greek',
   'Sp

In [24]:
list_game_detail_df=pd.DataFrame(list_game_detail)
list_game_detail_df.to_json('date1.json',orient="index",indent=2)
list_game_detail_df

,name,genre,description,feature_list,supported_languages,release_date,developer,publisher,num_curators,num_reviews,requirements
0,Counter-Strike 2,"[Action, Free to Play]","For over two decades, Counter-Strike has offer...","[Cross-Platform Multiplayer, Steam Trading Car...","[English, Czech, Danish, Dutch, Finnish, Frenc...","Aug 21, 2012",Valve,Valve,"13,562 Curators have reviewed this product. Cl...","[All (7,676,687)\nPositive (6,762,062)\nNegati...",[MINIMUM:\nOS: Windows® 10\nProcessor: 4 hardw...
1,Dota 2,"[Action, Strategy, Free to Play]","Every day, millions of players worldwide enter...","[Steam Trading Cards, Steam Workshop, SteamVR ...","[English, Bulgarian, Czech, Danish, Dutch, Fin...","Jul 9, 2013",Valve,Valve,"3,236 Curators have reviewed this product. Cli...","[All (2,074,538)\nPositive (1,703,810)\nNegati...",[MINIMUM:\nOS: Windows 7 or newer\nProcessor: ...
2,Call of Duty®,"[Adventure, RPG, Strategy]","Baldur’s Gate 3 is a story-rich, party-based R...","[Single-player, Online Co-op, LAN Co-op, Steam...","[English, French, German, Spanish - Spain, Pol...","Aug 3, 2023",Larian Studios,Larian Studios,550 Curators have reviewed this product. Click...,"[All (366,721)\nPositive (351,896)\nNegative (...",[MINIMUM:\nRequires a 64-bit processor and ope...
3,Apex Legends™,"[Action, Adventure, Free to Play]","Apex Legends is the award-winning, free-to-pla...","[Online PvP, Online Co-op, Steam Achievements,...","[English, French, Italian, German, Spanish - S...","Nov 5, 2020",Respawn Entertainment,Electronic Arts,933 Curators have reviewed this product. Click...,"[All (730,958)\nPositive (583,474)\nNegative (...",[MINIMUM:\nRequires a 64-bit processor and ope...
4,Team Fortress 2,"[Action, Free to Play]",Nine distinct classes provide a broad range of...,"[Cross-Platform Multiplayer, Steam Achievement...","[English, Danish, Dutch, Finnish, French, Germ...","Oct 10, 2007",Valve,Valve,"8,581 Curators have reviewed this product. Cli...","[All (996,230)\nPositive (935,357)\nNegative (...",[MINIMUM:\nOS: Windows® 7 (32/64-bit)/Vista/XP...
...,...,...,...,...,...,...,...,...,...,...,...
94,Paladins®,"[Action, Adventure, Massively Multiplayer, Str...","Join 50+ million players in Paladins, the free...","[Online PvP, Steam Achievements, Full controll...","[English, French, German, Spanish - Spain, Pol...","May 8, 2018",Evil Mojo Games,Hi-Rez Studios,"1,020 Curators have reviewed this product. Cli...","[All (340,107)\nPositive (289,873)\nNegative (...",[MINIMUM:\nOS: Windows 7 64-bit or newer\nProc...
95,American Truck Simulator,"[Indie, Simulation]",Experience legendary American trucks and deliv...,"[Single-player, Online Co-op, Steam Achievemen...","[English, French, Italian, German, Spanish - S...","Feb 2, 2016",SCS Software,SCS Software,716 Curators have reviewed this product. Click...,"[All (136,452)\nPositive (131,906)\nNegative (...",[MINIMUM:\nOS: Windows 7 64-bit\nProcessor: In...
96,Crab Game,"[Action, Casual, Free to Play, Indie, Racing]",Crab Game is a First-Person Multiplayer game w...,"[Online PvP, Online Co-op]",[English],"Oct 29, 2021",Dani,Dani,272 Curators have reviewed this product. Click...,"[All (148,643)\nPositive (136,892)\nNegative (...",[MINIMUM:\nRequires a 64-bit processor and ope...
97,Halo Infinite,"[Action, Free to Play]",The legendary Halo series returns with the mos...,"[Online PvP, LAN PvP, Cross-Platform Multiplay...","[English, French, Italian, German, Spanish - S...","Nov 15, 2021",343 Industries,Xbox Game Studios,446 Curators have reviewed this product. Click...,"[All (162,495)\nPositive (113,669)\nNegative (...",[MINIMUM:\nRequires a 64-bit processor and ope...


In [26]:
for i in range(0,99):
  if len(list_game_detail_df.iloc[i].genre)==0:
    print(i)